# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [29]:
pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [68]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [69]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [70]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [71]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_69126d60e6008191b1aaf8747068121b


### Upload Files

In [72]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-1aoLecAwM5zpsync4yk9yw


### Attach File to Vector Store

In [73]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-1aoLecAwM5zpsync4yk9yw


### Query the Vector Store

In [74]:
query = "the latest development in generativeAI"

In [75]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

## OpenAI Response API

### Simple Response

In [38]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [39]:
display(Markdown(simple_response.output_text))

As of the latest updates in 2023, some key developments in generative AI include:

1. **Multimodal Models**: Models that can process and generate multiple types of data, such as text, images, and audio. OpenAI's GPT-4 and similar models have paved the way for more integrated, versatile systems.

2. **Improved Efficiency**: Advances in reducing the computational requirements of generative models, making them more accessible and environmentally friendly. Techniques like model pruning and quantization are being widely adopted.

3. **Enhanced Creativity**: Generative AI is being used in new creative domains, from video game design to music composition, allowing for more intricate and interactive outputs.

4. **AI in Research and Problem Solving**: These models are being applied in scientific research, from protein folding predictions to drug discovery, showcasing their potential beyond traditional data generation.

5. **Ethical and Bias Mitigation**: There is a strong focus on addressing biases in AI outputs and making systems more transparent, fair, and accountable.

6. **Real-time Collaboration Tools**: Development of AI systems that can collaborate with humans in real-time, aiding in brainstorming and creative processes.

7. **Augmented Reality Integration**: Generative AI is being integrated with augmented reality technologies to create immersive experiences, impacting industries like education and entertainment.

These trends highlight the ongoing innovation in the field and its expanding impact across various sectors.

### File Search Response

In [40]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [41]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis using generative AI, SAP BTP, and AWS Bedrock to assess risk and boost resilience.

3. **Market Growth**: The global generative AI market is projected to reach $1.18 billion this year, highlighting the rapid growth and adoption of these technologies.

4. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises with tools for secure and responsible deployment, allowing teams to build custom large language models.

5. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of cinematic content, such as food commercials, without the need for traditional production setups.

These developments illustrate the expanding role of generative AI across various sectors, from enterprise solutions to creative content generation.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [42]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [43]:
display(Markdown(web_search_response.output_text))

Here are the **most significant recent developments (as of November 10, 2025)** in the generative AI arena, highlighting key milestones across models, tools, integrations, and ecosystem advancements:

---

**1. OpenAI Rolls Out Sora for Android**  
- **Launch & Adoption**: OpenAI’s text-to-video app *Sora* debuted on Android on **November 6, 2025**, achieving a remarkable **~470,000 first-day downloads**, significantly outperforming its iOS debut (~360,000 downloads) ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).  
- **Significance**: This rapid adoption underscores a growing mainstream appetite for AI-powered video generation—Sora is expanding access to creative tools that were once niche.

---

**2. Google Expands Gemini Ecosystem with DS STAR, Deep Research, Chrome AI Mode & Nano Banana 2**  
- **DS STAR**: Google introduced a new multi-agent framework—**DS STAR (Data Science Agent via Iterative Planning and Verification)**—capable of converting ambiguous business problems into executable Python code across mixed data formats (CSV, JSON, Markdown, unstructured text) ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).  
- **Deep Research**: A novel Gemini feature that harnesses user’s Gmail, Drive, and Chat content to generate detailed, context-aware research reports ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).  
- **Chrome AI Mode**: Google added an AI-mode button in Chrome (iOS & Android) to streamline AI-powered search access, initially in the U.S., soon across 160 countries supporting multiple languages ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).  
- **Nano Banana 2 Incoming**: A successor to the viral *Nano Banana* (Gemini 2.5 Flash Image), internally branded “GEMPIX2,” is set to launch imminently—suggesting an imminent leap in Google’s image-generation capabilities ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

---

**3. Advances in Generative AI Tooling & Ecosystems**  
- **Orchestration & Automation**: Platforms in 2025 increasingly adopt orchestration frameworks to manage multiple generative models within cohesive pipelines—featuring dynamic routing, context sharing, and error handling—for improved scalability and reliability ([futureagi.com](https://futureagi.com/blogs/generative-ai-trends-2025?utm_source=openai)).  
- **Open-Source Expansion**: The democratization of generative AI is accelerating, with widespread adoption of tools like Stable Diffusion WebUI and Hugging Face Transformers making it easier for developers to build and deploy models ([futureagi.com](https://futureagi.com/blogs/generative-ai-trends-2025?utm_source=openai)).

---

**4. Industry Programs & Strategic Support Initiatives**  
- **Meta’s Llama Startup Program**: Meta launched a program offering early-stage startups up to **$6,000/month** in cloud inference credits (for six months) and technical assistance, boosting generative AI innovation among emerging developers ([medium.com](https://medium.com/%40ivanreznikov/may-2025-generative-ai-landscape-pushing-the-needle-f5d3cb7e2422?utm_source=openai)).  
- **AWS GAIA Accelerator**: Amazon Web Services selected **40 startups** for its 2025 Generative AI Accelerator (GAIA) cohort. This eight-week program offers mentorship and resources in foundational model development, AI tooling, and infrastructure across multiple global regions ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/aws-selects-40-startups-for-2025-generative-ai-accelerator-program/articleshow/124377434.cms?utm_source=openai)).

---

**5. Model Highlights Across Leading AI Developers**  
- **GPT‑5 (OpenAI)**: Released on **August 7, 2025**, GPT‑5 is OpenAI’s latest multimodal foundation model, integrated into ChatGPT and Microsoft Copilot, offering enhanced reasoning capabilities and enterprise performance. It underpins services for roughly 700 million users ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).  
- **Runway Gen‑4**: A text-to-video model launched on **March 31, 2025**. While capable of generating short (up to 10-second) clips using text prompts and reference images, it still faces limitations around character consistency and motion artifacts ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai)).  
- **Grok 3 (xAI)**: Elon Musk’s xAI introduced **Grok 3** on **February 17, 2025**, trained on expansive compute and datasets—claimed to outperform GPT‑4o on reasoning benchmarks. It also added image editing, API access, and a “Think” mode. Despite controversy, it remains available to users ([en.wikipedia.org](https://en.wikipedia.org/wiki/Grok_%28chatbot%29?utm_source=openai)).

---

**Summary of Trends & Context**  
Across the generative AI landscape, the **focus has shifted** from isolated breakthroughs to integrated, user-ready solutions:

- **Mainstream adoption**: Tools like Sora and Gemini’s new features are embedding AI into everyday applications (video, search, productivity).  
- **Ecosystem investment**: Companies are investing in developer tools (orchestration, open-source libraries) and sponsoring emerging players via programs like AWS GAIA and Meta’s Llama initiative.  
- **Model evolution**: From GPT‑5 to Grok 3 and Runway Gen‑4, the emphasis is on reasoning, multimodal generation, and accessibility.

---

Please let me know if you'd like deeper analysis on any of these announcements or want to explore emerging generative AI use cases, technologies, or implications in specific industries!

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [44]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here are the **most significant recent developments (as of November 10, 2025)** in the generative AI

### Continue Query with Web Search

In [45]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [46]:
display(Markdown(continue_search_response.output_text))

Here’s a refreshed and diverse roundup of the **latest developments in generative AI** as of early November 2025, sourced from recent news, research, and product releases. Each section is clearly cited and structured to provide depth and context.

---

##  Recent Major Product & Model Announcements

• **OpenAI Launches ‘Aardvark’ Agentic Security Researcher**  
  On October 30, OpenAI announced *Aardvark*, a GPT‑5 powered autonomous agent in private beta capable of detecting software vulnerabilities. It achieves approximately 92 % recall on seeded repositories and has already been responsible for 10 CVE‑tracked disclosures ([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai)).

• **Adobe Expands Firefly’s Generative Capabilities**  
  At Adobe MAX 2025, Adobe unveiled new AI features in its Firefly suite: tools to generate soundtracks and speech with customizable emotional tone, support for training custom models using as few as six images, and a browser-based, multi-track AI-powered video editor. They also introduced “Project Moonlight” to bridge user context across creative apps ([wired.com](https://www.wired.com/story/adobe-max-2025-firefly-photoshop-updates?utm_source=openai)).

• **Engineers Introduce Google’s “Nano Banana 2” (GEMPIX2)**  
  Google is preparing to launch *Nano Banana 2*, internally named *GEMPIX2*, as announced in the Gemini ecosystem. An image-generation model expected to arrive within weeks, it builds on the viral success of the original *Nano Banana* ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

---

##  Infrastructure & Hardware Breakthroughs

• **OpenAI’s $38B AWS Cloud Deal**  
  On November 4, OpenAI entered a landmark $38 billion multi-year agreement with Amazon Web Services for cloud infrastructure, breaking its previous exclusivity with Microsoft. The deal secures large-scale compute resources, including GPUs and storage, for training next-gen AI models ([jasonwade.com](https://www.jasonwade.com/today-s-ai-tech-news-november-4-2025?utm_source=openai)).

• **US to Host DOE’s Largest AI Supercomputer**  
  NVIDIA and Oracle revealed plans to deploy the Department of Energy’s biggest AI supercomputer at Argonne National Laboratory. The system, named *Solstice*, will include 100,000 NVIDIA Blackwell GPUs and deliver a combined 2,200 exaflops, aimed at accelerating scientific AI research ([champaignmagazine.com](https://champaignmagazine.com/2025/11/02/ai-by-ai-weekly-top-5-october-27-november-2-2025/?utm_source=openai)).

---

##  Generative AI Adoption & Ecosystem Trends

• **Explosive Use of GenAI in Steam Games**  
  A study shows that one in five Steam games released in 2025 incorporates generative AI—a 681 % year-on-year surge. Out of roughly 114,000 titles, 7,818 disclose GenAI usage, primarily in visual assets, but also in audio, narrative, and live gameplay elements ([tomshardware.com](https://www.tomshardware.com/video-games/pc-gaming/1-in-5-steam-games-released-in-2025-use-generative-ai-up-nearly-700-percent-year-on-year-7-818-titles-disclose-genai-asset-usage-7-percent-of-entire-steam-library?utm_source=openai)).

---

##  Research & Long-Term Vision

• **Gemini 2.5’s “Historic” AI Breakthrough**  
  Google DeepMind’s Gemini 2.5 model achieved a milestone by earning a gold medal at the ICPC for solving complex programming challenges faster than all but one human competitor. The feat was hailed as a “profound leap” toward AGI, showcasing advanced abstraction and problem-solving, although not without scrutiny over compute resources ([theguardian.com](https://www.theguardian.com/technology/2025/sep/17/google-deepmind-claims-historic-ai-breakthrough-in-problem-solving?utm_source=openai)).

• **Generative AI’s Effect on Information Diversity**  
  A recent study finds that autonomous AI-generated articles can increase content diversity in otherwise homogeneous news ecosystems but may reduce it when diversity is already high. These findings shape how we weigh AI’s influence on democratic discourse ([arxiv.org](https://arxiv.org/abs/2503.16021?utm_source=openai)).

---

##  Other Noteworthy Tidbits

• **Adobe’s AI Expansion Across Creative Suite** — See the “Infrastructure & Hardware” section above for details on Firefly, custom models, and workflow integrations.

• **Google Maps & Chrome Add AI Features** — Gemini is being embedded into services like Google Maps (as a voice assistant) and Chrome (via an AI mode button), reinforcing AI’s seamless integration into everyday tools ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

---

###  Context & Summary

These developments collectively indicate a significant shift in the generative AI landscape:

- **Autonomy in tools**: AI systems like *Aardvark* are moving beyond assistance to proactive agents.
- **Creative realism**: Adobe and Google are refining generative content with emotional fidelity, context-awareness, and richer media editing.
- **Compute scaling**: Enormous infrastructure deals and supercomputer deployments indicate industry-wide commitment to AI compute at scale.
- **Mainstream usage**: GenAI is permeating domains like gaming, where it now contributes to a substantial portion of new titles.
- **Research milestones**: Breakthroughs like Gemini 2.5’s performance at ICPC underscore advancements toward AGI, balanced by academic analyses of AI’s societal impact.

---

If you'd like a deeper dive into any of these updates—such as technical breakdowns, competitive comparisons (e.g., Aardvark vs. existing tools), or implications for your interests—let me know!

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [47]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [48]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include several key advancements and applications:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis tool using generative AI, SAP BTP, and AWS Bedrock to assess risk and enhance resilience.

3. **Market Growth**: The global generative AI market is projected to reach $1.18 billion this year, highlighting the rapid growth and adoption of these technologies.

4. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of cinematic videos and other content with minimal human intervention.

5. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing for the development of custom large language models to improve customer engagement and streamline processes.

These developments indicate a broadening scope of generative AI applications across various sectors, from supply chain management to creative content production and enterprise solutions.

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [76]:
with open('crime matrix.pdf', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-S3aLDwXLrHy862XENQK4j8


In [77]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)


file-S3aLDwXLrHy862XENQK4j8


In [86]:
query2 = "what is the analysis of this crime report"

In [87]:
search_results = client.vector_stores.search(
    vector_store_id = vector_store_id,
    query=query2
)
for result in search_results.data[:5]:
    print(result.content[0].text[:100]+"\n Relevant score: "+str(result.score))



Bundy took pleasure from these individuals’ pain and the sense of control 
he felt even after they 
 Relevant score: 0.03225806451612903
"tweet": {
    "text": "RT @myexamcloud: Humans and Software Developers Futures Using and Developing
 Relevant score: 0.03177805800756621
Later that day, 17-year-old Debra Kent disappears after leaving a high school play 
to pick up her 
 Relevant score: 0.03125763125763126
IA 280 Crime Matrix: Ted Bundy
Ted Bundy Killings: A Timeline of His Twisted Reign of Terror
The i
 Relevant score: 0.03055037313432836
He is extradited to Aspen, Colorado, in 
early 1977 to stand trial.
June 7, 1977: Bundy escapes fr
 Relevant score: 0.029138513513513514


In [47]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [47]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [91]:
display(Markdown(simple_response.output_text))

Please provide the details of the crime report you'd like me to analyze, and I'll be happy to help!